In [1]:
import numpy as np

In [2]:
W = 1   #Width 
K = 1   #Depth
n = 3
dimensions = 3
def atomic_distances(pos, output, L=1, W=1): # output = 0 gives the relative distances, output = 1 gives the relative positions
    rel_pos = pos[:, None, :] - pos[None, :, :]                 # returns one matrix for each particle. Relative distances within the box
    rel_dist = np.zeros((n, n, 1))
    for i in range(n):
        for k in range(n):
            for l in range(dimensions):
                if l == 0: # x-values i.e. change by L
                    rel_pos[i,k,l] = min(rel_pos[i,k,l], (rel_pos[i,k,l]+L)%L, (rel_pos[i,k,l]-L)%L, rel_pos[i,k,l]+L, rel_pos[i,k,l]-L, key=abs) # takes the smallest distance comparing all images
                if l == 1:
                    rel_pos[i,k,l] = min(rel_pos[i,k,l], (rel_pos[i,k,l]+W)%W, (rel_pos[i,k,l]-W)%W, rel_pos[i,k,l]+W, rel_pos[i,k,l]-W, key=abs)
                if l == 2:
                    rel_pos[i,k,l] = min(rel_pos[i,k,l], (rel_pos[i,k,l]+K)%K, (rel_pos[i,k,l]-K)%K, rel_pos[i,k,l]+K, rel_pos[i,k,l]-K, key=abs)
    for i in range(n):    
        for k in range(n):
            rel_dist[i,k] = np.sqrt(np.dot(rel_pos[i,k,:],rel_pos[i,k,:]))
    if output == 0:
        return rel_dist
    elif output == 1:
        return rel_pos
    else:
        print("The second parameter in the lj_forces must be either 0(for distances) or 1(for positions)")

In [3]:
#compute the forces on the particles at each timestep, pos = [m]/[s] = 
def lj_force(position):
    rel_pos = atomic_distances(position[:,-3:], 1) # calculates the relative position of the particles, with pos = np.array(N_particels, 3)
    n = np.shape(rel_pos)[0] # number of particles
    r = np.sqrt(np.sum((rel_pos)**2, axis=2)) # n x n simmetric matrix, r[i,j] is the distance between the i-th and the j-th particles
    R = np.zeros((n,n,3))
    for i in range(n):
        for j in range (n):
            for k in range (3):
                R[i,j,k] = r [i,j]
                if r[i,j] == 0:
                    R[i,j,k] = 1
    # R are n matricies in the form nx3, where the i-th matrix contains the distances from the i-th particle, calling R_i the i-th matrix, 
    # R_i[j,k] gives the distance between the i-th and the j-th particles and is the same value for all the 'k in range(3)', i.e. on all the j-th line
    # in addition, R_i[i,k] (i-th line of the i-th matrix) is equal to 1 instead of 0 in order not to divide by 0 in the computation of F
    # we can do this since rel_pos_i[i,k] multiplies everything in F and is equal to 0
    F = np.zeros((n,n,3))
    F = rel_pos*R
    F = 24*rel_pos*(((1/R)**12)-((1/R)**6))
    F_matrix = np.sum(F, axis=0)  
    return F_matrix # The output is an nx3 matrix

In [4]:
A = np.array([[0.26,0.37,0.81],[0.45,0.62,0.94],[0.75,0.53,0.38],[0.54,0.38,0.82],[0.13,0.26,0.87]])
lj_force(A)

array([[-2.56876105e+09, -2.19575278e+09,  1.20194328e+09],
       [ 6.22474149e+06, -2.43724412e+07, -1.22035268e+07],
       [-2.87130173e+04, -8.43050984e+03,  1.78990433e+04],
       [-3.68874056e+07,  2.07200458e+07,  9.87766479e+06],
       [ 2.59945243e+09,  2.19941360e+09, -1.19963532e+09]])